In [ ]:
!pip install ipyleaflet

In [ ]:
from IPython import display
from PIL import Image, ExifTags
from ipyleaflet import Map , Marker

Metadata is data that provides information about other data. It's often described as "data about data". For example, a book's metadata might include its title, author, publication date, and genre. For an image, metadata could include the camera model, date taken, and location.

In [ ]:
display.Image("/content/image1.jpg" , width=560 , height=430)

##**EXIF Files**
EXIF (Exchangeable Image File Format) is a standard that specifies formats for images, sound, and ancillary tags used by digital cameras, scanners, and other systems handling image and sound files recorded by digital cameras. It stores metadata about the image, such as camera model, date and time of capture, exposure settings, GPS location, and other camera-specific information.

In [ ]:
def get_exif(filename):
    exif_data = {}
    image = Image.open(filename)
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = ExifTags.TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for gps_tag in value:
                    sub_decoded = ExifTags.GPSTAGS.get(gps_tag, gps_tag)
                    gps_data[sub_decoded] = value[gps_tag]
                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data

source:[Extract GPS data using python and PIL is failing ](https://stackoverflow.com/questions/72530975/extract-gps-data-using-python-and-pil-is-failing)from answer on StackOverflow.com

In [ ]:
exif_data = get_exif("/content/image1.jpg")

In [ ]:
print(exif_data)

##**Extracting the Date and Time a Picture was Taken**


In [ ]:
exif_data['DateTime']

##**Extracting Decimal Degree Format to Map Image**
Decimal degrees (DD) is a notation for expressing latitude and longitude geographic coordinates as decimal fractions of a degree. DD are used in many geographic information systems (GIS), web mapping applications such as OpenStreetMap, and GPS devices.

In [ ]:
def gps_extract (exif_dict):
  gps_metadata = exif_dict['GPSInfo']

  lat_ref_num = 0
  if gps_metadata['GPSLatitudeRef'] == 'N':
   lat_ref_num += 1
  if gps_metadata['GPSLatitudeRef'] == 'S':
   lat_ref_num -= 1

  lat_list = [float(num) for num in gps_metadata['GPSLatitude']]
  lat_coordiante = (lat_list[0] + (lat_list[1] / 60) + (lat_list[2] / 3600)) * lat_ref_num

  #longitudinal information
  #positive longitudes are east of the prime meridian, negative longitudes are west of the prime meridian
  long_ref_num = 0
  if gps_metadata['GPSLongitudeRef'] == 'E':
    long_ref_num += 1
  if gps_metadata['GPSLongitudeRef'] == 'W':
    long_ref_num -= 1

  long_list = [float(num) for num in gps_metadata['GPSLongitude']]
  long_coordiante = (long_list[0]+long_list[1]/60+long_list[2]/3600) * long_ref_num


  #return the latitude and longitude as a tuple
  return (lat_coordiante,long_coordiante)





In [ ]:
gps_extract(exif_data)

##**Plotting the Location with `ipyleaflet`**

In [ ]:
center = gps_extract(exif_data)

m = Map(center=center, zoom=15)

marker = Marker(location=center, draggable=False)
m.add_layer(marker);

m

##**Image Files Missing Metadata**

In [ ]:
display.Image('/content/image2.png',width=512,height=313)

In [ ]:
get_exif("/content/image2.png")

##**Removing Exif Data from Image**

In [ ]:
#exif data can be removed by reading a picture to pillow's Image class
#then a copy of the picture can be saved or the original can be overwritten
#from the Image class

img = Image.open('/content/image1.jpg')
img.save('image1_no_metadata.jpg')
get_exif('/content/image1_no_metadata.jpg')

In [ ]:
display.Image('/content/image1_no_metadata.jpg',width=504,height=672)